In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
ROOT_PATH = '../..'

In [3]:
import pandas as pd
import numpy as np

dataset95 = pd.read_csv(f'{ROOT_PATH}/datasets/openfoodfacts.csv', delimiter='\t', usecols=['origins', 'manufacturing_places', 'countries'])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
fr = ['France', 'en:FR', 'France,Europe','Belgique,France']
dataset95[dataset95['manufacturing_places'].notnull()][dataset95['origins'].notnull()][dataset95['countries'].notnull()]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,origins,manufacturing_places,countries
362,France,France,France
365,Québec,Brossard Québec,Canada
381,Québec,Brossard Québec,Canada
404,France,United Kingdom,United Kingdom
415,Québec,Brossard Québec,Canada
...,...,...,...
2874163,France,"France,Normandie,Calvados",France
2874164,"France,Normandie","France,Normandie,Calvados",France
2874165,"Agen,France","Agropole,Estillac",France
2874178,Sweden,"Filipstad,Suède","France,Spain"


In [ ]:
# dataset95[dataset95['countries_ingredients'] == 'Norway']
# dataset95[dataset95['countries_manufacturing'] == 'France'].count()

### Occurences of different colums

In [ ]:
N = len(dataset95)

fr_with_nutrition = len(dataset95[dataset95['nutrition-score-fr_100g'].apply(lambda x: x.is_integer())])
print(f'FR with nutrition: {fr_with_nutrition} | {format(100*fr_with_nutrition/N, ".1f")}%')

uk_with_nutrition = len(dataset95[dataset95['nutrition-score-uk_100g'].apply(lambda x: x.is_integer())])
print(f'UK with nutrition: {uk_with_nutrition} | {format(100*uk_with_nutrition/N, ".1f")}%')

carbon_footprint = len(dataset95[dataset95['carbon-footprint-from-meat-or-fish_100g'].apply(lambda x: x.is_integer())])
print(f'Carbon footprint: {carbon_footprint} | {format(100*carbon_footprint/N, ".1f")}%')


### The brands which occur the most often in the dataset

In [ ]:
# Split the 'brands' column by ',' and stack the resulting DataFrames
split_brands = dataset95['brands'].str.split(',', expand=True).stack()

# Remove leading/trailing whitespaces
split_brands = split_brands.str.strip()

# Count the occurrences of each brand
brand_counts = split_brands.value_counts()

print(brand_counts[:40])

### Type of packaging used

In [ ]:
# count packaging used
country_tags = ['fr', 'es']
packaging = {}
for ps in dataset95['packaging_en']:
  ps = str(ps).split(',')
  for p in ps:
    if p[:2] in country_tags:
      continue
    packaging[p] = packaging.get(p, 0) + 1
    
# transform to tuple and sort
packaging = [(k, v) for k,v in packaging.items()]
packaging.sort(key=lambda x: x[1], reverse=True)

packaging[:30]

In [ ]:
# Define the target brand
target_brand = "Nestlé"

country_tags = ['fr', 'es', 'ar']

# Split the comma-separated brand and country strings into lists
dataset95['brands_list'] = dataset95['brands'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
dataset95['countries_list'] = dataset95['countries_en'].apply(lambda x: x.split(',') if isinstance(x, str) else [])

# Create a boolean mask for rows containing the target brand
mask_brand = dataset95['brands_list'].apply(lambda brands: target_brand in brands)

# Filter the DataFrame using the mask
filtered_dataset95 = dataset95[mask_brand]

# Create an empty dictionary to store the occurrences of each country
country_counts = {}

# Iterate through the filtered DataFrame
for _, row in filtered_dataset95.iterrows():
    for country in row['countries_list']:
        if country[:2] in country_tags:
            continue
        if country not in country_counts:
            country_counts[country] = 1
        else:
            country_counts[country] += 1

# sort
sorted_country_counts = dict(sorted(country_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Occurrences of each country for the brand '{target_brand}':")
# print(country_counts)
for k,v in sorted_country_counts.items():
    print(f'{k}: {v}')

In [ ]:
dataset95['ecoscore_grade']